# PROJET DE FIN D'ETUDE
## THEME: ANALYSE DES SENTIMENTS POUR LA SATISFACTION CLIENT
### AUTEUR: Herman Nketchemen
### IAS-M2-JV-DA1

### Plan de travail

Contexte
Problematique


### Chargemenet des bibliothèques

In [2]:
import requests
import pandas as pd
import io
import gzip


In [3]:
# chargement du dataset

df = pd.read_csv("../data/Reviews.csv")
df.head()


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
df.shape

(568454, 10)

In [5]:
selected = ["Id", "Score", "Summary", "Text"]
reviews = df[selected]

In [6]:
# Affichage des 20 premières lignes de notre dataset
reviews.head(20)

,Id,Score,Summary,Text
0,1,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,4,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,2,Cough Medicine,If you are looking for the secret ingredient i...
4,5,5,Great taffy,Great taffy at a great price. There was a wid...
5,6,4,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,5,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,5,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,5,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,5,Healthy Dog Food,This is a very healthy dog food. Good for thei...
